# 第5课：异步编程

## 学习目标
- 理解异步编程的概念
- 掌握 async/await 语法
- 学会使用 asyncio 库
- 了解异步 I/O 操作

## 1. 异步编程概念

异步编程允许程序在等待 I/O 操作时执行其他任务，提高效率。

- **同步**：一个任务完成后才能开始下一个
- **异步**：任务可以暂停，让其他任务运行

## 2. async/await 基础

In [ ]:
import asyncio
import time

# 定义协程函数
async def say_hello(name, delay):
    print(f"开始: {name}")
    await asyncio.sleep(delay)  # 异步等待
    print(f"完成: {name}")
    return f"Hello, {name}!"

# 在 Jupyter 中运行协程
result = await say_hello("World", 1)
print(result)

In [ ]:
# 并发执行多个协程
async def main():
    start = time.time()
    
    # 并发执行
    results = await asyncio.gather(
        say_hello("Alice", 2),
        say_hello("Bob", 1),
        say_hello("Charlie", 3)
    )
    
    print(f"\n结果: {results}")
    print(f"总耗时: {time.time() - start:.2f}秒")

await main()

## 3. Task 和并发控制

In [ ]:
# 创建任务
async def fetch_data(url, delay):
    print(f"开始获取: {url}")
    await asyncio.sleep(delay)
    return f"数据来自 {url}"

async def main():
    # 创建任务
    task1 = asyncio.create_task(fetch_data("api/users", 2))
    task2 = asyncio.create_task(fetch_data("api/posts", 1))
    
    # 等待所有任务完成
    result1 = await task1
    result2 = await task2
    
    print(f"结果1: {result1}")
    print(f"结果2: {result2}")

await main()

In [ ]:
# 使用 wait 和超时
async def slow_task(name, delay):
    await asyncio.sleep(delay)
    return f"{name} 完成"

async def main():
    tasks = [
        asyncio.create_task(slow_task("A", 1)),
        asyncio.create_task(slow_task("B", 3)),
        asyncio.create_task(slow_task("C", 2))
    ]
    
    # 等待第一个完成的任务
    done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
    
    print(f"已完成: {[t.result() for t in done]}")
    print(f"未完成: {len(pending)} 个任务")
    
    # 取消未完成的任务
    for task in pending:
        task.cancel()

await main()

In [ ]:
# 带超时的操作
async def long_operation():
    await asyncio.sleep(10)
    return "完成"

async def main():
    try:
        result = await asyncio.wait_for(long_operation(), timeout=2.0)
        print(result)
    except asyncio.TimeoutError:
        print("操作超时！")

await main()

## 4. 异步迭代器和生成器

In [ ]:
# 异步生成器
async def async_range(start, end):
    for i in range(start, end):
        await asyncio.sleep(0.5)
        yield i

async def main():
    async for num in async_range(1, 5):
        print(f"数字: {num}")

await main()

In [ ]:
# 异步上下文管理器
class AsyncResource:
    async def __aenter__(self):
        print("获取资源...")
        await asyncio.sleep(0.5)
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print("释放资源...")
        await asyncio.sleep(0.5)
    
    async def do_something(self):
        print("使用资源")

async def main():
    async with AsyncResource() as resource:
        await resource.do_something()

await main()

## 5. Semaphore 限制并发

In [ ]:
# 限制同时运行的任务数
async def download(url, semaphore):
    async with semaphore:
        print(f"下载: {url}")
        await asyncio.sleep(1)
        print(f"完成: {url}")
        return url

async def main():
    semaphore = asyncio.Semaphore(3)  # 最多同时 3 个
    # 注意：以下 URL 仅为示例，不会实际访问
    urls = [f"http://example.com/file{i}" for i in range(10)]
    
    tasks = [download(url, semaphore) for url in urls]
    results = await asyncio.gather(*tasks)
    
    print(f"\n下载了 {len(results)} 个文件")

await main()

## 6. 异步队列

In [ ]:
# 生产者-消费者模式
async def producer(queue, n):
    for i in range(n):
        await asyncio.sleep(0.5)
        await queue.put(f"item-{i}")
        print(f"生产: item-{i}")

async def consumer(queue, name):
    while True:
        item = await queue.get()
        if item is None:
            break
        print(f"{name} 消费: {item}")
        await asyncio.sleep(0.3)
        queue.task_done()

async def main():
    queue = asyncio.Queue()
    
    # 启动生产者和消费者
    producer_task = asyncio.create_task(producer(queue, 5))
    consumers = [asyncio.create_task(consumer(queue, f"Consumer-{i}")) for i in range(2)]
    
    # 等待生产者完成
    await producer_task
    
    # 等待队列处理完
    await queue.join()
    
    # 停止消费者
    for _ in consumers:
        await queue.put(None)
    
    await asyncio.gather(*consumers)

await main()

## 7. 实际应用示例

In [ ]:
# 模拟异步 HTTP 请求
import random

async def fetch_url(url):
    """模拟异步 HTTP 请求（不会实际访问网络）"""
    delay = random.uniform(0.5, 2.0)
    await asyncio.sleep(delay)
    return {"url": url, "status": 200, "delay": delay}

async def main():
    # 注意：以下 URL 仅为示例，模拟请求不会实际访问
    urls = [
        "https://api.example.com/users",
        "https://api.example.com/posts",
        "https://api.example.com/comments",
        "https://api.example.com/albums",
        "https://api.example.com/photos"
    ]
    
    start = time.time()
    
    # 并发请求所有 URL
    results = await asyncio.gather(*[fetch_url(url) for url in urls])
    
    for result in results:
        print(f"{result['url']}: {result['delay']:.2f}s")
    
    print(f"\n总耗时: {time.time() - start:.2f}秒")

await main()

## 8. 练习题

### 练习：异步文件下载器
实现一个模拟的异步文件下载器，限制同时下载数量

In [ ]:
async def download_file(filename, size_mb):
    """模拟下载文件"""
    # 在这里编写代码
    pass

async def download_all(files, max_concurrent=3):
    """下载所有文件，限制并发数"""
    # 在这里编写代码
    pass

# 测试
files = [
    ("file1.zip", 100),
    ("file2.zip", 50),
    ("file3.zip", 200),
    ("file4.zip", 80),
    ("file5.zip", 150)
]

## 9. 本课小结

1. **async/await**：定义和调用协程
2. **asyncio.gather**：并发执行多个协程
3. **asyncio.create_task**：创建任务
4. **Semaphore**：限制并发数量
5. **Queue**：异步生产者-消费者模式
6. **异步迭代器/上下文管理器**：async for、async with